In [56]:
from zhinst.toolkit import Session, CommandTable, Waveforms
import random
import numpy as np
from IPython.display import display, clear_output
import time

import matplotlib.pyplot as plt

dataserver_ip = "10.42.11.4"
dataserver_port = 8004
device_id = "dev12135"

session = Session(dataserver_ip, dataserver_port)
device = session.connect_device(device_id)
fs = 2e9

device.system.preset.load(1) 
device.system.preset.load.wait_for_state_change(0)

In [57]:
ct = CommandTable(device.sgchannels[0].awg.commandtable.load_validation_schema())
# display(device.sgchannels[0].awg.commandtable.load_validation_schema())

In [64]:
def upload_ct_v0(rate_divider):
    ct = CommandTable(device.sgchannels[0].awg.commandtable.load_validation_schema())
    ct.table[0].waveform.index = 0 # index in assignWaveIndex
    ct.table[0].amplitude00.value = 1
    ct.table[0].amplitude01.value = -1
    ct.table[0].amplitude10.value = 1
    ct.table[0].amplitude11.value = 1
    ct.table[0].waveform.samplingRateDivider = rate_divider
    device.sgchannels[0].awg.commandtable.upload_to_device(ct)


def upload_v0_wave(v0_amp, vset_pulse_len_samples):
    wave01 = np.ones(vset_pulse_len_samples) * v0_amp
    waveforms = Waveforms() 
    waveforms.assign_waveform(
        slot= 0, # index in assignWaveIndex
        wave1= wave01,
        wave2= wave01)
    device.sgchannels[0].awg.write_to_waveform_memory(waveforms)

def upload_ct_vset(rate_divider):
    ct = CommandTable(device.sgchannels[1].awg.commandtable.load_validation_schema())
    ct.table[0].waveform.index = 0 # index in assignWaveIndex
    ct.table[0].amplitude00.value = 1
    ct.table[0].amplitude01.value = -1
    ct.table[0].amplitude10.value = 1
    ct.table[0].amplitude11.value = 1
    ct.table[0].waveform.samplingRateDivider = rate_divider
    device.sgchannels[1].awg.commandtable.upload_to_device(ct)

def upload_vset_wave(vset_amp, v0_pulse_len_samples):
    wave01 = np.ones(v0_pulse_len_samples)* vset_amp
    waveforms = Waveforms() 
    waveforms.assign_waveform(
        slot= 0, # index in assignWaveIndex
        wave1= wave01,
        wave2= wave01)
    device.sgchannels[1].awg.write_to_waveform_memory(waveforms)

def get_results(result_node, timeout):
    wave_data_captured = {}
    wave_data_captured[result_node] = False
    start_time = time.time()
    captured_data = {}
    while not all(wave_data_captured.values()):
        if start_time + timeout < time.time():
            raise TimeoutError('Timeout before all samples collected.')
        test = session.poll()
        for node, value in test.items():
            node = session.raw_path_to_node(node)
            for v in value:
                if node not in captured_data:
                    captured_data[node] = [v['vector']]
                else:
                    captured_data[node].append(v['vector'])
            if len(captured_data[node]) >= 1: # readout 1 point
                wave_data_captured[node] = True
                total_num_data = sum([len(element) for element in captured_data[node]])
    data = captured_data[result_node][0]
    return data

In [66]:
ro_f = 100e6
ro_amp = 0.2
int_len = 100e-6
wait_after_int = 10e-6

num_avgs = 1
num_points = 10

rate_divider = 4

v0_amp = 0
v0_amps = np.linspace(v0_amp, 0.3, num_points)
v0_pulse_len = 100e-6
v0_pulse_len_samples = int(int(v0_pulse_len*fs/2**rate_divider/16)*16)

vset_amp = 0
vset_pulse_len_samples = v0_pulse_len_samples


qa_seqc = f'''
    repeat({num_avgs}){{
        waitDigTrigger(1);
        setTrigger(1);
        playZero({int((int_len + wait_after_int)*1e6)}, AWG_RATE_976KHZ);
        setTrigger(0);
    }}
'''
sg_seqc = f''' 
    assignWaveIndex(placeholder({v0_pulse_len_samples}), placeholder({v0_pulse_len_samples}), 0);
    repeat({num_avgs}){{
        waitDigTrigger(1);
        executeTableEntry(0);
    }}
'''

with device.set_transaction():
    device.qachannels[0].output.rflfpath(0) # set LF mode
    device.qachannels[0].input.rflfpath(0) # set LF mode

    device.qachannels[0].oscs[0].freq(ro_f) # set readout frequency 
    device.qachannels[0].oscs[0].gain(ro_amp) # set gain factor: 0 to 1 
    device.qachannels[0].generator.load_sequencer_program(qa_seqc) # load seqc
    device.qachannels[0].generator.enable(0) # disable generator 
    device.qachannels[0].generator.auxtriggers[0].channel(8)

    device.qachannels[0].spectroscopy.delay(220e-9) # integration delay in units of second
    device.qachannels[0].spectroscopy.length(int(int_len*fs)) # integration length in units of number of samples
    device.qachannels[0].spectroscopy.result.averages(num_avgs) # number of averages 
    device.qachannels[0].spectroscopy.result.enable(0) # disable result logger

    device.qachannels[0].input.range(0)
    device.qachannels[0].output.range(-5)  
    device.qachannels[0].output.on(1)
    device.qachannels[0].input.on(1)

    for i in range(2):
        device.sgchannels[i].output.rflfpath(0) # set ch1 and ch2 to LF mode
        device.synthesizers[1].centerfreq(0)
        device.sgchannels[i].output.range(-5)
        device.sgchannels[i].awg.load_sequencer_program(sg_seqc) # load seqc
        device.sgchannels[i].awg.auxtriggers[0].channel(8)
        if i == 0:
            upload_ct_v0(rate_divider)
            upload_v0_wave(v0_amp,vset_pulse_len_samples)
        else:
            upload_ct_vset(rate_divider)
            upload_v0_wave(vset_amp, vset_pulse_len_samples)

        device.sgchannels[i].oscs[0].freq(0)
        device.sgchannels[i].output.on(1)


fd_vset = np.zeros(num_points)

results = np.zeros(num_points, dtype = np.complex128)
for i, v0_amp in enumerate(v0_amps):
    result_node = device.qachannels[0].spectroscopy.result.data.wave   
    result_node.subscribe()
    upload_v0_wave(v0_amp, v0_pulse_len_samples)
    upload_vset_wave(vset_amp, vset_pulse_len_samples)
    device.qachannels[0].spectroscopy.result.enable(1) # enable result logger
    device.qachannels[0].generator.enable(1) # enable generator
    device.sgchannels[0].awg.enable(1)
    device.sgchannels[1].awg.enable(1)

    device.system.internaltrigger.holdoff(int_len + wait_after_int)
    device.system.internaltrigger.repetitions(num_avgs)
    device.system.internaltrigger.enable(1)
    results[i] = get_results(result_node, 1) # get results 
    result_node.unsubscribe()
    vset_amp = vset_amp + 0.1


C:\Users\chunyans\AppData\Local\Temp\ipykernel_6072\2239553334.py:89: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  results[i] = get_results(result_node, 1) # get results
